# RAG from scratch

- Adapted from https://github.com/opendatahub-io/llama-stack-demos/
- Requires distributions/masterclass-agents/run.yaml

In [8]:
import rich
from llama_stack_client import LlamaStackClient, RAGDocument


In [9]:
client = LlamaStackClient(
    base_url="http://localhost:8321",
)

In [10]:
client.models.list()

[Model(identifier='all-MiniLM-L6-v2', metadata={'embedding_dimension': 384.0}, api_model_type='embedding', provider_id='ollama', provider_resource_id='all-minilm:latest', type='model', model_type='embedding'),
 Model(identifier='meta-llama/Llama-3.2-3B-Instruct', metadata={}, api_model_type='llm', provider_id='ollama', provider_resource_id='llama3.2:3b-instruct-fp16', type='model', model_type='llm')]

In [11]:
vector_db_id = "my-vector-db"
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model="all-MiniLM-L6-v2",
    embedding_dimension=384,
)

VectorDBRegisterResponse(embedding_dimension=384, embedding_model='all-MiniLM-L6-v2', identifier='my-vector-db', provider_id='milvus', provider_resource_id='my-vector-db', type='vector_db', access_attributes=None)

Ingesting documents into a vector database:

In [12]:
urls = ["chat.rst", "llama3.rst", "memory_optimizations.rst", "lora_finetune.rst"]
documents = [
    RAGDocument(
        document_id=f"num-{i}",
        content=f"https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/{url}",
        mime_type="text/plain",
        metadata={},
    )
    for i, url in enumerate(urls)
]


In [13]:
rich.print(documents)

[
    {
        'document_id': 'num-0',
        'content': 'https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/chat.rst',
        'mime_type': 'text/plain',
        'metadata': {}
    },
    {
        'document_id': 'num-1',
        'content': 'https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/llama3.rst',
        'mime_type': 'text/plain',
        'metadata': {}
    },
    {
        'document_id': 'num-2',
        'content': 
'https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/memory_optimizations.rst',
        'mime_type': 'text/plain',
        'metadata': {}
    },
    {
        'document_id': 'num-3',
        'content': 
'https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/lora_finetune.rst',
        'mime_type': 'text/plain',
        'metadata': {}
    }
]

In [14]:

client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=512,
)

In [25]:
prompt = "What are the top 5 topics that were explained? Only list succinct bullet points."

In [16]:
# higher level tool provides packaged results, can span multiple dbs
tool_response = client.tool_runtime.rag_tool.query(
    content=prompt, vector_db_ids=[vector_db_id]
)
rich.print(tool_response)

QueryResult(
    metadata={'document_ids': ['num-3', 'num-0', 'num-2', 'num-2', 'num-2']},
    content=[
        TextContentItem(
            text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n',
            type='text'
        ),
        TextContentItem(
            text='Result 1:\nDocument_id:num-3\nContent: ,\n    and (b) the memory constraints of your 
hardware.\n\nThe preceding command will run a LoRA finetune with torchtune\'s factory settings, but we may want to 
experiment a bit.\nLet\'s take a closer look at some of the :code:`lora_finetune_distributed` config.\n\n.. 
code-block:: yaml\n\n  # Model Arguments\n  model:\n    _component_: lora_llama2_7b\n    lora_attn_modules: 
[\'q_proj\', \'v_proj\']\n    lora_rank: 8\n    lora_alpha: 16\n  ...\n\nWe see that the default is to apply LoRA 
to Q and V projections with a rank of 8.\nSome experiments with LoRA have found that it can be beneficial to apply 
LoRA to all linear layers in\nthe self-attention, and to increase the rank to 16 or 32. Note that this is likely to
increase our max memory,\nbut as long as we keep :code:`rank<<embed_dim`, the impact should be relatively 
minor.\n\nLet\'s run this experiment. We can also increase alpha (in general it is good practice to scale alpha and
rank together).\n\n.. code-block:: bash\n\n    tune run --nnodes 1 --nproc_per_node 2 lora_finetune_distributed 
--config llama2/7B_lora \\\n    lora_attn_modules=[\'q_proj\',\'k_proj\',\'v_proj\',\'output_proj\'] \\\n    
lora_rank=32 lora_alpha=64 output_dir=./lora_experiment_1\n\nA comparison of the (smoothed) loss curves between 
this run and our baseline over the first 500 steps can be seen below.\n\n.. image:: 
/_static/img/lora_experiment_loss_curves.png\n\n.. note::\n    The above figure was generated with W&B. You can use
torchtune\'s :class:`~torchtune.training.metric_logging.WandBLogger`\n    to generate similar loss curves, but you 
will need to install W&B and setup an account separately. For more details on\n    using W&B in torchtune, see our 
":ref:`wandb_logging`" recipe.\n\n.. _lora_tutorial_memory_tradeoff_label:\n\nTrading off memory and model 
performance with LoRA\n--------------------------------------------------\n\nIn the preceding example, we ran LoRA 
on two devices. But given LoRA\'s low memory footprint, we can run fine-tuning\non a single device using most 
commodity GPUs which support `bfloat16 <https://\n',
            type='text'
        ),
        TextContentItem(
            text='Result 2:\nDocument_id:num-0\nContent:  Instruct.\n\n.. 
_prompt_template_vs_special_tokens:\n\nTokenizing prompt templates & special 
tokens\n--------------------------------------------\n\nLet\'s say I have a sample of a single user-assistant turn 
accompanied with a system\nprompt:\n\n.. code-block:: python\n\n    sample = [\n        {\n            "role": 
"system",\n            "content": "You are a helpful, respectful, and honest assistant.",\n        },\n        {\n 
"role": "user",\n            "content": "Who are the most influential hip-hop artists of all time?",\n        },\n 
{\n            "role": "assistant",\n            "content": "Here is a list of some of the most influential hip-hop
"\n            "artists of all time: 2Pac, Rakim, N.W.A., Run-D.M.C., and Nas.",\n        },\n    ]\n\nNow, let\'s 
format this with the :class:`~torchtune.models.llama2.Llama2ChatTemplate` class and\nsee how it gets tokenized. The
Llama2ChatTemplate is an example of a **prompt template**,\nwhich simply structures a prompt with flavor text to 
indicate a certain task.\n\n.. code-block:: python\n\n    from torchtune.data import Llama2ChatTemplate, 
Message\n\n    messages = [Message.from_dict(msg) for msg in sample]\n    formatted_messages = 
Llama2ChatTemplate.format(messages)\n    print(formatted_messages)\n    # [\n    #     Message(\n    #         
role=\'user\',\n    #         content=\'[INST] <<SYS>>\\nYou are a helpful, respectful, and hon

In [17]:
# we can also query the vector db directly
db_response = client.vector_io.query(
    vector_db_id=vector_db_id,
    query=prompt,
)
rich.print(db_response)

QueryChunksResponse(
    chunks=[
        Chunk(
            content=',\n    and (b) the memory constraints of your hardware.\n\nThe preceding command will run a 
LoRA finetune with torchtune\'s factory settings, but we may want to experiment a bit.\nLet\'s take a closer look 
at some of the :code:`lora_finetune_distributed` config.\n\n.. code-block:: yaml\n\n  # Model Arguments\n  model:\n
_component_: lora_llama2_7b\n    lora_attn_modules: [\'q_proj\', \'v_proj\']\n    lora_rank: 8\n    lora_alpha: 
16\n  ...\n\nWe see that the default is to apply LoRA to Q and V projections with a rank of 8.\nSome experiments 
with LoRA have found that it can be beneficial to apply LoRA to all linear layers in\nthe self-attention, and to 
increase the rank to 16 or 32. Note that this is likely to increase our max memory,\nbut as long as we keep 
:code:`rank<<embed_dim`, the impact should be relatively minor.\n\nLet\'s run this experiment. We can also increase
alpha (in general it is good practice to scale alpha and rank together).\n\n.. code-block:: bash\n\n    tune run 
--nnodes 1 --nproc_per_node 2 lora_finetune_distributed --config llama2/7B_lora \\\n    
lora_attn_modules=[\'q_proj\',\'k_proj\',\'v_proj\',\'output_proj\'] \\\n    lora_rank=32 lora_alpha=64 
output_dir=./lora_experiment_1\n\nA comparison of the (smoothed) loss curves between this run and our baseline over
the first 500 steps can be seen below.\n\n.. image:: /_static/img/lora_experiment_loss_curves.png\n\n.. note::\n   
The above figure was generated with W&B. You can use torchtune\'s 
:class:`~torchtune.training.metric_logging.WandBLogger`\n    to generate similar loss curves, but you will need to 
install W&B and setup an account separately. For more details on\n    using W&B in torchtune, see our 
":ref:`wandb_logging`" recipe.\n\n.. _lora_tutorial_memory_tradeoff_label:\n\nTrading off memory and model 
performance with LoRA\n--------------------------------------------------\n\nIn the preceding example, we ran LoRA 
on two devices. But given LoRA\'s low memory footprint, we can run fine-tuning\non a single device using most 
commodity GPUs which support `bfloat16 <https://',
            metadata={'token_count': 512.0, 'document_id': 'num-3'}
        ),
        Chunk(
            content=' Instruct.\n\n.. _prompt_template_vs_special_tokens:\n\nTokenizing prompt templates & special 
tokens\n--------------------------------------------\n\nLet\'s say I have a sample of a single user-assistant turn 
accompanied with a system\nprompt:\n\n.. code-block:: python\n\n    sample = [\n        {\n            "role": 
"system",\n            "content": "You are a helpful, respectful, and honest assistant.",\n        },\n        {\n 
"role": "user",\n            "content": "Who are the most influential hip-hop artists of all time?",\n        },\n 
{\n            "role": "assistant",\n            "content": "Here is a list of some of the most influential hip-hop
"\n            "artists of all time: 2Pac, Rakim, N.W.A., Run-D.M.C., and Nas.",\n        },\n    ]\n\nNow, let\'s 
format this with the :class:`~torchtune.models.llama2.Llama2ChatTemplate` class and\nsee how it gets tokenized. The
Llama2ChatTemplate is an example of a **prompt template**,\nwhich simply structures a prompt with flavor text to 
indicate a certain task.\n\n.. code-block:: python\n\n    from torchtune.data import Llama2ChatTemplate, 
Message\n\n    messages = [Message.from_dict(msg) for msg in sample]\n    formatted_messages = 
Llama2ChatTemplate.format(messages)\n    print(formatted_messages)\n    # [\n    #     Message(\n    #         
role=\'user\',\n    #         content=\'[INST] <<SYS>>\\nYou are a helpful, respectful, and honest 
assistant.\\n<</SYS>>\\n\\nWho are the most influential hip-hop artists of all time? [/INST] \',\n    #         
...,\n    #     ),\n    #     Message(\n    #         role=\'assistant\',\n    #         content=\'Here is a list 
of some of the most influential hip-hop artists of all

In [18]:
# prompt_context = tool_response.content
prompt_context = "\n".join([c.content for c in db_response.chunks])

In [26]:
messages = [{"role": "system", "content": "You are a helpful assistant."}]
extended_prompt = f"""
Please answer the given query using the context below.

QUERY:
{prompt}

CONTEXT:
{prompt_context}
"""
messages.append({"role": "user", "content": extended_prompt})
rich.print(messages)

[
    {'role': 'system', 'content': 'You are a helpful assistant.'},
    {
        'role': 'user',
        'content': '\nPlease answer the given query using the context below.\n\nQUERY:\nWhat are the top 5 topics 
that were explained? Only list succinct bullet points.\n\nCONTEXT:\n,\n    and (b) the memory constraints of your 
hardware.\n\nThe preceding command will run a LoRA finetune with torchtune\'s factory settings, but we may want to 
experiment a bit.\nLet\'s take a closer look at some of the :code:`lora_finetune_distributed` config.\n\n.. 
code-block:: yaml\n\n  # Model Arguments\n  model:\n    _component_: lora_llama2_7b\n    lora_attn_modules: 
[\'q_proj\', \'v_proj\']\n    lora_rank: 8\n    lora_alpha: 16\n  ...\n\nWe see that the default is to apply LoRA 
to Q and V projections with a rank of 8.\nSome experiments with LoRA have found that it can be beneficial to apply 
LoRA to all linear layers in\nthe self-attention, and to increase the rank to 16 or 32. Note that this is likely to
increase our max memory,\nbut as long as we keep :code:`rank<<embed_dim`, the impact should be relatively 
minor.\n\nLet\'s run this experiment. We can also increase alpha (in general it is good practice to scale alpha and
rank together).\n\n.. code-block:: bash\n\n    tune run --nnodes 1 --nproc_per_node 2 lora_finetune_distributed 
--config llama2/7B_lora \\\n    lora_attn_modules=[\'q_proj\',\'k_proj\',\'v_proj\',\'output_proj\'] \\\n    
lora_rank=32 lora_alpha=64 output_dir=./lora_experiment_1\n\nA comparison of the (smoothed) loss curves between 
this run and our baseline over the first 500 steps can be seen below.\n\n.. image:: 
/_static/img/lora_experiment_loss_curves.png\n\n.. note::\n    The above figure was generated with W&B. You can use
torchtune\'s :class:`~torchtune.training.metric_logging.WandBLogger`\n    to generate similar loss curves, but you 
will need to install W&B and setup an account separately. For more details on\n    using W&B in torchtune, see our 
":ref:`wandb_logging`" recipe.\n\n.. _lora_tutorial_memory_tradeoff_label:\n\nTrading off memory and model 
performance with LoRA\n--------------------------------------------------\n\nIn the preceding example, we ran LoRA 
on two devices. But given LoRA\'s low memory footprint, we can run fine-tuning\non a single device using most 
commodity GPUs which support `bfloat16 <https://\n Instruct.\n\n.. 
_prompt_template_vs_special_tokens:\n\nTokenizing prompt templates & special 
tokens\n--------------------------------------------\n\nLet\'s say I have a sample of a single user-assistant turn 
accompanied with a system\nprompt:\n\n.. code-block:: python\n\n    sample = [\n        {\n            "role": 
"system",\n            "content": "You are a helpful, respectful, and honest assistant.",\n        },\n        {\n 
"role": "user",\n            "content": "Who are the most influential hip-hop artists of all time?",\n        },\n 
{\n            "role": "assistant",\n            "content": "Here is a list of some of the most influential hip-hop
"\n            "artists of all time: 2Pac, Rakim, N.W.A., Run-D.M.C., and Nas.",\n        },\n    ]\n\nNow, let\'s 
format this with the :class:`~torchtune.models.llama2.Llama2ChatTemplate` class and\nsee how it gets tokenized. The
Llama2ChatTemplate is an example of a **prompt template**,\nwhich simply structures a prompt with flavor text to 
indicate a certain task.\n\n.. code-block:: python\n\n    from torchtune.data import Llama2ChatTemplate, 
Message\n\n    messages = [Message.from_dict(msg) for msg in sample]\n    formatted_messages = 
Llama2ChatTemplate.format(messages)\n    print(formatted_messages)\n    # [\n    #     Message(\n    #         
role=\'user\',\n    #         content=\'[INST] <<SYS>>\\nYou are a helpful, respectful, and honest 
assistant.\\n<</SYS>>\\n\\nWho are the most influential hip-hop artists of all time? [/INST] \',\n    #         
...,\n    #     ),\n    #     Message(\n    #         role=\'assista

In [27]:
response = client.inference.chat_completion(
    messages=messages,
    model_id="meta-llama/Llama-3.2-3B-Instruct",
)
rich.print(response)

ChatCompletionResponse(
    completion_message=CompletionMessage(
        content='Here are the top 5 topics that were explained:\n\n* LoRA (Low-Rank Adaptation)\n* QLoRA (Quantized
Low-Rank Adaptation)\n* DoRA (Weight-Decomposed Low-Rank Adaptation)\n* Llama2ChatTemplate (prompt template for 
structured prompts)\n* Memory constraints and trade-offs in fine-tuning models with LoRA',
        role='assistant',
        stop_reason='end_of_turn',
        tool_calls=[]
    ),
    logprobs=None,
    metrics=[
        Metric(metric='prompt_tokens', value=1592.0, unit=None),
        Metric(metric='completion_tokens', value=85.0, unit=None),
        Metric(metric='total_tokens', value=1677.0, unit=None)
    ]
)

In [34]:
# Left as an extra exercise for the reader
from llama_stack_client import Agent, AgentEventLogger

rag_agent = Agent(
    client, 
    model="meta-llama/Llama-3.2-3B-Instruct",
    instructions="You are a helpful assistant",
    tools = [
        {
          "name": "builtin::rag/knowledge_search",
          "args" : {
            "vector_db_ids": [vector_db_id],
          }
        }
    ],
)